# ADB Phase 2 Project Evaluation Notebook


**Purpose**: This notebook evaluates the performance of a semantic search project by analyzing databases of various sizes.

### Evaluation Focus:
- **Database Sizes**:
  - 1 Million Records
  - 10 Million Records
  - 15 Million Records
  - 20 Million Records

For each database size, this notebook will:
- Generate random vectors for the database.
- Use the `VecDB` class (implemented by students) to retrieve queries
- Evaluate and report retrieval time, accuracy, and RAM usage.

### Project Constraints:
Refer to the project document for details on RAM, Disk, Time, and Score constraints.

### Notebook Structure:
1. **Part 1 - Modifiable Cells**:
   - Includes cells that teams are allowed to modify, specifically for these variables only:
     - GitHub repository link (including PAT token).
     - Google Drive IDs for indexes files.
     - Paths for loading existing indexes.

2. **Part 2 - Non-Modifiable Cells**:
   - Contains essential setup and evaluation code that must not be modified.
   - Students should only modify inputs in Part 1 to ensure smooth execution of the notebook.

## Part 1 - Modifiable Cells

Each team must provide a unique GitHub repository link that includes a PAT token. This link will allow the notebook to download the necessary code for evaluation.

In [238]:
# !git clone https://github.com/farah-moh/vec_db.git

# Database Path Instructions


Teams need to specify paths for each database (1M, 10M, 15M, 20M records) as follows:

1. Zip each database directory/file after generation.
2. Upload the zip file to Google Drive.
3. Share the file with "Anyone with the link."
4. Extract the file ID from the link (e.g., for `https://drive.google.com/file/d/1j1gAU3kvdRqcOoKI5K5FgMMUZpOQANah/view`, the ID is `1j1gAU3kvdRqcOoKI5K5FgMMUZpOQANah`).
5. Assign each ID to the appropriate variable in Part 1.
6. Provide the local PATH for each database to be passed to the initializer for automatic loading of the database and index (to be submitted during the project final phase).

**Note**: The code will download and unzip these files automatically. Once extracted, the local path for each database should be specified to enable the notebook to load databases and indexes.

In [239]:
TEAM_NUMBER = 14
# GDRIVE_ID_DB_1M = "1XbEP6sU0k0UbuPcQLkHJP7cdPtebpsbD"
# GDRIVE_ID_DB_10M = "1Jho9rair77eWdA5-iI_qDT2spJNesuDe"
# GDRIVE_ID_DB_15M = "1sPIgNxIuNDUUBnTvAuPMLryd1mqmgwV3"
# GDRIVE_ID_DB_20M = "1j1gAU3kvdRqcOoKI5K5FgMMUZpOQANah"
PATH_DB_1M = "saved_db_1m.csv"
PATH_DB_10M = "saved_db_10m.csv"
PATH_DB_15M = "saved_db_15m.csv"
PATH_DB_20M = "saved_db_20m.csv"

**Query Seed Number**:
This number will be adjusted during discussions by the instructor.


In [240]:
QUERY_SEED_NUMBER =50

**Final Submission Checklist**:
Ensure the following items are included in your final submission:
- `TEAM_NUMBER`
- GitHub clone link (with PAT token)
- Google Drive IDs for each database:
  - `GDRIVE_ID_DB_1M`, `GDRIVE_ID_DB_10M`, `GDRIVE_ID_DB_15M`, `GDRIVE_ID_DB_20M`
- Paths for each database:
  - `PATH_DB_1M`, `PATH_DB_10M`, `PATH_DB_15M`, `PATH_DB_20M`
- Project document detailing the work and findings.

## Part 2: Do Not Modify Beyond This Point
### Note:
This section contains setup and evaluation code that should not be edited by students. Only the instructor may modify this section in case of a major bug.


In [241]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [242]:
# %cd vec_db

This cell to run any additional requirement that your code need <br>


In [243]:
!pip install memory-profiler >> log.txt
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


This cell to download the zip files and unzip them here.

In [244]:
# !gdown $GDRIVE_ID_DB_1M -O saved_db_1m.zip
# !gdown $GDRIVE_ID_DB_10M -O saved_db_10m.zip
# !gdown $GDRIVE_ID_DB_15M -O saved_db_15m.zip
# !gdown $GDRIVE_ID_DB_20M -O saved_db_20m.zip
# !unzip saved_db_1m.zip
# !unzip saved_db_10m.zip
# !unzip saved_db_15m.zip
# !unzip saved_db_20m.zip

These are the functions for running and reporting

In [245]:
import numpy as np
DB_SEED_NUMBER = 42
ELEMENT_SIZE = np.dtype(np.float32).itemsize
DIMENSION = 70

In [246]:
import numpy as np
import os
from vec_db import VecDB
import time
from dataclasses import dataclass
from typing import List
from memory_profiler import memory_usage
import gc

@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]

def run_queries(db, queries, top_k, actual_ids, num_runs):
    """
    Run queries on the database and record results for each query.

    Parameters:
    - db: Database instance to run queries on.
    - queries: List of query vectors.
    - top_k: Number of top results to retrieve.
    - actual_ids: List of actual results to evaluate accuracy.
    - num_runs: Number of query executions to perform for testing.

    Returns:
    - List of Result
    """
    global results
    results = []
    for i in range(num_runs):
        tic = time.time()
        db_ids = db.retrieve(queries[i], top_k)
        toc = time.time()
        run_time = toc - tic
        results.append(Result(run_time, top_k, db_ids, actual_ids[i]))
    return results

def memory_usage_run_queries(args):
    """
    Run queries and measure memory usage during the execution.

    Parameters:
    - args: Arguments to be passed to the run_queries function.

    Returns:
    - results: The results of the run_queries.
    - memory_diff: The difference in memory usage before and after running the queries.
    """
    global results
    mem_before = max(memory_usage())
    mem = memory_usage(proc=(run_queries, args, {}), interval = 1e-3)
    return results, max(mem) - mem_before

def evaluate_result(results: List[Result]):
    """
    Evaluate the results based on accuracy and runtime.
    Scores are negative. So getting 0 is the best score.

    Parameters:
    - results: A list of Result objects

    Returns:
    - avg_score: The average score across all queries.
    - avg_runtime: The average runtime for all queries.
    """
    scores = []
    run_time = []
    for res in results:
        run_time.append(res.run_time)
        # case for retireving number not equal to top_k, socre will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            scores.append( -1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    score -= ind
            except:
                score -= len(res.actual_ids)
        scores.append(score)

    return sum(scores) / len(scores), sum(run_time) / len(run_time)

def get_actual_ids_first_k(actual_sorted_ids, k):
    """
    Retrieve the IDs from the sorted list of actual IDs.
    actual IDs has the top_k for the 20 M database but for other databases we have to remove the numbers higher than the max size of the DB.

    Parameters:
    - actual_sorted_ids: A list of lists containing the sorted actual IDs for each query.
    - k: The DB size.

    Returns:
    - List of lists containing the actual IDs for each query for this DB.
    """
    return [[id for id in actual_sorted_ids_one_q if id < k] for actual_sorted_ids_one_q in actual_sorted_ids]

This code to generate all the files for databases.

In [247]:
def _write_vectors_to_file(vectors: np.ndarray, db_path) -> None:
    mmap_vectors = np.memmap(db_path, dtype=np.float32, mode='w+', shape=vectors.shape)
    mmap_vectors[:] = vectors[:]
    mmap_vectors.flush()

def get_all_rows(db_path) -> np.ndarray:
    vectors = np.memmap(db_path, dtype=np.float32, mode='r', shape=(10**6, DIMENSION))
    return np.array(vectors)


def generate_database(size: int) -> None:
    rng = np.random.default_rng(DB_SEED_NUMBER)
    vectors = rng.random((size, DIMENSION), dtype=np.float32)
    return vectors

# vectors = get_all_rows('saved_db_1M.dat')
vectors = generate_database(20*10**6)

db_filename_size_20M = 'saved_db_20M.dat'
if not os.path.exists(db_filename_size_20M): _write_vectors_to_file(vectors, db_filename_size_20M)
db_filename_size_15M = 'saved_db_15M.dat'
if not os.path.exists(db_filename_size_15M): _write_vectors_to_file(vectors[:15*10**6], db_filename_size_15M)
db_filename_size_10M = 'saved_db_10M.dat'
if not os.path.exists(db_filename_size_10M): _write_vectors_to_file(vectors[:10*10**6], db_filename_size_10M)
db_filename_size_1M = 'saved_db_1M.dat'
if not os.path.exists(db_filename_size_1M): _write_vectors_to_file(vectors[:1*10**6], db_filename_size_1M)


Code to generate the queries that will be used to evaluate the questions.

Note: QUERY_SEED_NUMBER will be changed at submission day

In [248]:
needed_top_k = 10000
rng = np.random.default_rng(QUERY_SEED_NUMBER)
query1 = rng.random((1, 70), dtype=np.float32)
query2 = rng.random((1, 70), dtype=np.float32)
query3 = rng.random((1, 70), dtype=np.float32)
query_dummy = rng.random((1, 70), dtype=np.float32)

actual_sorted_ids_20m_q1 = np.argsort(vectors.dot(query1.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query1)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]
gc.collect()
actual_sorted_ids_20m_q2 = np.argsort(vectors.dot(query2.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query2)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]
gc.collect()
actual_sorted_ids_20m_q3 = np.argsort(vectors.dot(query3.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query3)), axis= 1).squeeze().tolist()[::-1][:needed_top_k]
gc.collect()

queries = [query1, query2, query3]
actual_sorted_ids_20m = [actual_sorted_ids_20m_q1, actual_sorted_ids_20m_q2, actual_sorted_ids_20m_q3]

In [249]:
# No more need to the actual vectors so delete it
del vectors
gc.collect()

0

This code to actually run the class you have been implemented. The `VecDB` class should take the database path, and index path that you provided.<br>
Note at the submission I'll not run the insert records. <br>
The query istelf will be changed at submissions day but not the DB

In [250]:
!ls -lh /content/level1_centroids_saved_db_1m.csv
!du -sh /content/second_level_centroids_saved_db_1m.csv
!ls -lh /content/level1_centroids_saved_db_10m.csv
!du -sh /content/second_level_centroids_saved_db_10m.csv
!ls -lh /content/level1_centroids_saved_db_15m.csv
!du -sh /content/second_level_centroids_saved_db_15m.csv

-rw-r--r-- 1 root root 71K Dec  8 17:45 /content/level1_centroids_saved_db_1m.csv
8.6M	/content/second_level_centroids_saved_db_1m.csv
-rw-r--r-- 1 root root 711K Dec  8 17:36 /content/level1_centroids_saved_db_10m.csv
78M	/content/second_level_centroids_saved_db_10m.csv
-rw-r--r-- 1 root root 1023K Dec  8 17:19 /content/level1_centroids_saved_db_15m.csv
136M	/content/second_level_centroids_saved_db_15m.csv


In [1]:
from vec_db import VecDB
db1 = VecDB(db_size = 20000000, database_file_path = "saved_db_20M.dat", index_file_path = "saved_db_20m.csv", new_db = True)

Number of clusters: 4400


c:\Users\ahma3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
c:\Users\ahma3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1962: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 512 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
c:\Users\ahma3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)
c:\Users\ahma3\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1962: UserWarning: MiniBatchKMeans is known to have a 

OSError: [Errno 28] No space left on device

In [252]:
results = []
to_print_arr = []

In [255]:
print("Team Number", TEAM_NUMBER)
database_info = {
    "1M": {
        "database_file_path": db_filename_size_1M,
        "index_file_path": PATH_DB_1M,
        "size": 10**6
    },
    "10M": {
        "database_file_path": db_filename_size_10M,
        "index_file_path": PATH_DB_10M,
        "size": 10 * 10**6
    },
    "15M": {
        "database_file_path": db_filename_size_15M,
        "index_file_path": PATH_DB_15M,
        "size": 15 * 10**6
    },
    # "20M": {
    #     "database_file_path": db_filename_size_20M,
    #     "index_file_path": PATH_DB_20M,
    #     "size": 20 * 10**6
    # }
}

for db_name, info in database_info.items():
    db = VecDB(database_file_path = info["database_file_path"], index_file_path = info["index_file_path"], new_db = False)
    actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, info["size"])
    # Make a dummy run query to make everything fresh and loaded (wrap up)
    res = run_queries(db, query_dummy, 5, actual_ids, 1)
    # actual runs to evaluate
    res, mem = memory_usage_run_queries((db, queries, 5, actual_ids, 3))
    eval = evaluate_result(res)
    to_print = f"{db_name}\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
    print(to_print)
    to_print_arr.append(to_print)
    del db
    del actual_ids
    del res
    del mem
    del eval
    gc.collect()

Team Number 14
1M	score	-9.666666666666666	time	0.88	RAM	5.66 MB
10M	score	-18.0	time	4.73	RAM	6.19 MB
15M	score	-85.33333333333333	time	4.04	RAM	6.16 MB


In [254]:
print("Team Number", TEAM_NUMBER)
print("\n".join(to_print_arr))

Team Number 14
1M	score	-9.666666666666666	time	1.10	RAM	5.86 MB
10M	score	-18.0	time	3.90	RAM	6.11 MB
15M	score	-85.33333333333333	time	2.95	RAM	6.06 MB
